In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import re

In [34]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

def vv(row):
    X = row.split('-')
    return X[0]+X[1]+X[2]

def vv2(row):
    X = row.split('/')
    return X[0]

def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d


def convert_ar_characters(input_str):

    mapping = {
        'ك': 'ک',
        'گ':'گ',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)


def _multiple_replace(mapping, text):
    pattern = "|".join(map(ree.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

In [3]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n = path + '\Cleaned_Stocks_Holders_1399-07-21_From96.csv'
df = pd.read_csv(n)
#######
df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)
HolderData = df

In [56]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n1 = path + '\Stocks_Prices_1399-07-20' + '.csv'
df1 = pd.read_csv(n1)
############
df1['jalaliDate'] = df1['jalaliDate'].apply(vv)
df = df1
symbols = [ 'سپرده','هما','وهنر-پذيره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['name'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)
df = df.drop(df[(df.name == 'اتکای')&(df.close_price == 1000)].index)
df = df.drop_duplicates()
df = df.drop(df.loc[(df['volume'] == 0)].index).sort_values(by =['name','jalaliDate']).drop(columns = ['volume','quantity']).rename(columns = {'name':'symbol'})
df = DriveYearMonthDay(df)
col = 'symbol'
df[col] = df[col].apply(lambda x: convert_ar_characters(x))

In [64]:
PriceData = df

In [57]:
n2 = path + '\\balance sheet - 9811' + '.xlsx'
df2 = pd.read_excel(n2)
df2 = df2.iloc[:,[0,4,13,-7]]
df2.rename(columns={ df2.columns[0]: "symbol" ,df2.columns[1]: "date" ,df2.columns[2]: "BookValue" ,df2.columns[3] : "Capital"}, inplace = True)
df2['shrout'] = df2['Capital'] * 100
df2['Year'] = df2['date'].apply(vv2)
df2['Year'] = df2['Year'].astype(str)
df2 = df2.drop(columns = ['date','Capital'])
col = 'symbol'
df2[col] = df2[col].apply(lambda x: convert_ar_characters(x))

In [132]:
PriceData = df
PriceData = PriceData.merge(df2,on=['symbol','Year'],how = 'left')
PriceData[['BookValue','shrout']] = PriceData[['BookValue','shrout']].fillna(method='ffill')

In [133]:
PriceData = PriceData[['jalaliDate', 'date', 'symbol','close_price','shrout','BookValue']]
PriceData['date1'] = PriceData['date'].apply(vv4)
PriceData['date1'] = pd.to_datetime(PriceData['date1']) 
PriceData['week_of_year'] = PriceData['date1'].dt.week 
PriceData['year_of_year'] = PriceData['date1'].dt.year

In [135]:
def weekRet(g):
    g['weekRet'] = (g.close_price.iloc[-1] - g.close_price.iloc[0])/g.close_price.iloc[0] * 100
    return g

gg = PriceData.groupby(['symbol', 'year_of_year','week_of_year'])
d = gg.apply(weekRet)
#Don't Delete this cell

In [136]:
def yearRet(g):
    f = g.groupby(['year_of_year']).last().reset_index()
    f['yearRet'] = f['close_price'].pct_change(periods = 1)*100
    f = f[['year_of_year','symbol','week_of_year','yearRet']]
    g = g.merge(f,on = ['year_of_year','symbol','week_of_year'])
    return g

gg = d.groupby(['symbol', 'week_of_year'])
d2 = gg.apply(yearRet)

In [144]:
PriceData = d2.reset_index(drop = True)
PriceData = PriceData.sort_values(by = ['symbol','date'])

In [146]:
shrout = HolderData[['symbol','shrout','date','jalaliDate']].drop_duplicates()
fkey = zip(list(shrout.symbol),list(shrout.date))
mapingdict = dict(zip(fkey,shrout.shrout))
PriceData['shrout2'] = PriceData.set_index(['symbol','date']).index.map(mapingdict)
PriceData.loc[~PriceData.shrout2.isnull(),'shrout'] = PriceData.loc[~PriceData.shrout2.isnull()]['shrout2'] 
PriceData = PriceData.drop(columns = ['shrout2'])

In [148]:
path2 = r"G:\TseClient\Data adjusted"
index = pd.read_excel(path2 + '\IRX6XTPI0009.xls')[['<COL14>','<CLOSE>']].rename(columns = {'<COL14>':'jalaliDate','<CLOSE>':'Index'})
index['Market_return'] =  index['Index'].pct_change(periods = 1)*100
Index = PriceData.merge(index , on ='jalaliDate')
Index = Index[['jalaliDate', 'date','week_of_year', 'year_of_year', 'Index',
       'Market_return']].drop_duplicates().sort_values(by = ['date'])
def weekRet(g):
    g['MarketweekRet'] = (g.Index.iloc[-1] - g.Index.iloc[0])/g.Index.iloc[0] * 100
    return g

gg = Index.groupby(['year_of_year','week_of_year'])
WeekIndex = gg.apply(weekRet)
WeekIndex = WeekIndex[['year_of_year','week_of_year','MarketweekRet']]

In [149]:
PriceData = PriceData.merge(WeekIndex,on = ['year_of_year','week_of_year']).drop_duplicates().sort_values(by = ['symbol','date'])

In [156]:
PriceData['MarketCap']= PriceData['close_price'] * PriceData['shrout']/1e5
PriceData['MarketCap'] = PriceData['MarketCap'].astype(int)
PriceData['BookToMarket'] = PriceData['BookValue']/PriceData['MarketCap']
PriceData = PriceData.reset_index(drop= True)
PriceData.head()

,jalaliDate,date,symbol,close_price,shrout,BookValue,date1,week_of_year,year_of_year,weekRet,yearRet,MarketweekRet,MarketCap,BookToMarket
0,13900422,20110713,آ س پ,3100.0,20000000.0,573994.0,2011-07-13,28,2011,4.903226,NaN,-0.632147,620000,0.925797
1,13900425,20110716,آ س پ,3252.0,20000000.0,573994.0,2011-07-16,28,2011,4.903226,NaN,-0.632147,650400,0.882525
2,13900427,20110718,آ س پ,3264.0,20000000.0,573994.0,2011-07-18,29,2011,-4.656863,NaN,2.947585,652800,0.879280
3,13900428,20110719,آ س پ,3203.0,20000000.0,573994.0,2011-07-19,29,2011,-4.656863,NaN,2.947585,640600,0.896026
4,13900429,20110720,آ س پ,3147.0,20000000.0,573994.0,2011-07-20,29,2011,-4.656863,NaN,2.947585,629400,0.911970


In [161]:
def Factor(g):
    Large = g.loc[g['MarketCap'] >= g['MarketCap'].quantile(0.9)]['weekRet'].mean()
    Small = g.loc[g['MarketCap'] <= g['MarketCap'].quantile(0.1)]['weekRet'].mean()
    
    Value = g.loc[g['BookToMarket'] >= g['BookToMarket'].quantile(0.9)]['weekRet'].mean()
    Growth = g.loc[g['BookToMarket'] <= g['BookToMarket'].quantile(0.1)]['weekRet'].mean()
    
    Winner = g.loc[g['yearRet'] >= g['yearRet'].quantile(0.9)]['weekRet'].mean()
    Loser = g.loc[g['yearRet'] <= g['yearRet'].quantile(0.1)]['weekRet'].mean()

    g['Winner_Loser'] = Winner - Loser
    g['SMB'] = Small - Large
    g['HML'] = Value - Growth
    
    g = g[['year_of_year','week_of_year','SMB','HML','Winner_Loser','MarketweekRet']].drop_duplicates(['year_of_year','week_of_year'])
    return g

WL = PriceData.groupby(['year_of_year','week_of_year'])
Factors = WL.apply(Factor)
Factors = Factors.reset_index(drop = True).drop(0)


In [163]:
Factors.to_excel(path + '\Analyzed Data'+'\Factors.xlsx',index= False)

In [ ]:
Factors = pd.read_excel(path + '\Analyzed Data'+'\Factors.xlsx')

In [ ]:
HolderData
HolderData['date1'] = HolderData['date'].apply(vv4)
HolderData['date1'] = pd.to_datetime(HolderData['date1']) 
HolderData['week_of_year'] = HolderData['date1'].dt.week 
HolderData['Month_of_year'] = HolderData['date1'].dt.month 
HolderData['year_of_year'] = HolderData['date1'].dt.year

In [ ]:
ldates = HolderData.groupby(['symbol','year_of_year','Month_of_year','week_of_year']).last()
ldates = ldates.reset_index()[['symbol','date']]


In [ ]:
[1]*5

In [ ]:
fkey = zip(list(ldates.symbol),list(ldates.date))
mapingdict = dict(zip(fkey,[1]*(len(ldates.symbol)+1)))
HolderData['Last'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)

In [ ]:
WeeklyHolderData = HolderData.loc[HolderData.Last == 1]

In [ ]:
for t in ['SMB','HML','Winner_Loser','MarketweekRet']: 
    print(t)
    mapingdict = dict(zip(zip(list(Factors.year_of_year),list(Factors.week_of_year)),list(Factors[t])))
    WeeklyHolderData[t] = WeeklyHolderData.set_index(['year_of_year','week_of_year']).index.map(mapingdict)

In [ ]:
fkey = zip(list(d.symbol),list(d.date))
mapingdict = dict(zip(fkey,d.weekRet))
WeeklyHolderData['weekRet'] = WeeklyHolderData.set_index(['symbol','date']).index.map(mapingdict)

In [ ]:
d

In [ ]:
wg = WeekPriceData.groupby(['symbol'])
def FourFactor(g):
    try:
        y = 'weekRet'
        x = ['MarketweekRet','HML','SMB','Winner_Loser']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['4-Residual'] = g ['weekRet'] - (model.params[0] + model.params[1]*g['MarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def ThreeFactor(g):
    try:
        y = 'weekRet'
        x = ['MarketweekRet','HML','SMB']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['3-Residual'] = g ['weekRet'] - (model.params[0] + model.params[1]*g['MarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB'] ) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g
re = wg.apply(FourFactor)
re2 = wg.apply(ThreeFactor)
re['3-Residual'] = re2['3-Residual']
re = re.reset_index(drop=True).drop(columns = ['group_name'])

In [ ]:
df = HolderData 